In [0]:
!pip install OWSLib nvcl-kit xmltodict tabulate --quiet

# NVCL Dataservices Notebook - Get Datasets

This notebook demonstrates on how you can query NVCL Dataservices to obtain the list of datasets available

Webserver URL of different states:

https://nvclwebservices.csiro.au/geoserver/wfs <br/>
https://gs.geoscience.nsw.gov.au/geoserver/wfs <br/>
https://geology.information.qld.gov.au/geoserver/wfs <br/>
https://sarigdata.pir.sa.gov.au/geoserver/wfs <br/>
http://geossdi.dmp.wa.gov.au/services/wfs <br/>
https://www.mrt.tas.gov.au/web-services/wfs <br/>
http://geology.data.nt.gov.au/geoserver/wfs <br/>
http://geology.data.vic.gov.au/nvcl/wfs <br/>

In [0]:
from nvcl_kit.reader import NVCLReader 
from types import SimpleNamespace
param = SimpleNamespace()

param.WFS_URL = "http://geology.data.vic.gov.au/nvcl/wfs"

# URL of NVCL service
param.NVCL_URL = "http://geology.data.vic.gov.au/NVCLDataServices/"

reader = NVCLReader(param)

In [0]:
if not reader.wfs:
    print("ERROR!")

In [0]:
nvcl_id_list = reader.get_nvcl_id_list()

In [0]:
import pandas as pd
import xmltodict
import json

data = []
for nvcl_id in nvcl_id_list:
  response_str = reader.svc.get_dataset_collection(nvcl_id)
  dataset_obj = xmltodict.parse(response_str)["DatasetCollection"]['Dataset']
  for dataset_dict in (dataset_obj if type(dataset_obj) is list else [dataset_obj]):
    dataset_dict['nvclID'] = nvcl_id
    data.append(dataset_dict)

dataset_df = pd.DataFrame(data)

In [0]:
from google.colab import data_table
data_table.DataTable(dataset_df[['nvclID', 'DatasetID', 'boreholeURI', 'DatasetName']], include_index=False, num_rows_per_page=25)

,nvclId,DatasetID,boreholeURI,DatasetName
0,877270,c7bfb49c-2e85-4d31-97f1-e9e273e299d,http://geology.data.vic.gov.au/resource/featur...,877270_KIND168
1,962832,3532134a-52f4-409f-9fa6-ab32c8a8987,http://geology.data.vic.gov.au/resource/featur...,BEU162
2,962851,89add48b-c99e-435a-8439-9b7e5ac32aa,http://geology.data.vic.gov.au/resource/featur...,NBD119
3,962864,1c9425cf-3b71-4d6f-93b5-a3e3e53310d,http://geology.data.vic.gov.au/resource/featur...,MH002
4,962878,804754fa-6f7e-4528-805e-09e8d90b8f2,http://geology.data.vic.gov.au/resource/featur...,SPD150
5,962890,852e635b-49dc-490f-bd0b-947ecc9e7cd,http://geology.data.vic.gov.au/resource/featur...,DDH120
6,962895,5b88ed23-98c5-4c3b-af08-1f3a8e01590,http://geology.data.vic.gov.au/resource/featur...,WWD025
7,976430,7bf74ba1-190b-4abf-9362-16216dec78c,http://geology.data.vic.gov.au/resource/featur...,976430_STAVELY02_DD
8,976431,ef550072-3d82-4e32-9175-c21a56d1e87,http://geology.data.vic.gov.au/resource/featur...,976431_STAVELY04_0_to_54m_Sonic
9,976431,da12d45f-1ef8-4f79-b6d2-355693934c3,http://geology.data.vic.gov.au/resource/featur...,976431_STAVELY04_56_to_102m_DD


In [0]:
from tabulate import tabulate

print(tabulate(dataset_df[['nvclId', 'DatasetID', 'boreholeURI', 'DatasetName']], tablefmt='html'))

<table>
<tbody>
<tr><td style="text-align: right;"> 0</td><td style="text-align: right;">877270</td><td>c7bfb49c-2e85-4d31-97f1-e9e273e299d</td><td>http://geology.data.vic.gov.au/resource/feature/GSV/borehole/877270</td><td>877270_KIND168                  </td></tr>
<tr><td style="text-align: right;"> 1</td><td style="text-align: right;">962832</td><td>3532134a-52f4-409f-9fa6-ab32c8a8987</td><td>http://geology.data.vic.gov.au/resource/feature/GSV/borehole/962832</td><td>BEU162                          </td></tr>
<tr><td style="text-align: right;"> 2</td><td style="text-align: right;">962851</td><td>89add48b-c99e-435a-8439-9b7e5ac32aa</td><td>http://geology.data.vic.gov.au/resource/feature/GSV/borehole/962851</td><td>NBD119                          </td></tr>
<tr><td style="text-align: right;"> 3</td><td style="text-align: right;">962864</td><td>1c9425cf-3b71-4d6f-93b5-a3e3e53310d</td><td>http://geology.data.vic.gov.au/resource/feature/GSV/borehole/962864</td><td>MH002                  

In [0]:
from ipywidgets import Checkbox

dataset_cw = Checkbox(
    value = True,
    description = '',
    disabled = False,
    indent = False
)

dataset_cw

Checkbox(value=True, indent=False)

In [0]:
!pip install jupyterlab --quiet

!pip install "jupyterlab>=1.0" jupyterlab-dash==0.1.0a3

!jupyter labextension install jupyterlab-dash@0.1.0-alpha.3

     |████████████████████████████████| 1.2MB 6.6MB/s 
     |████████████████████████████████| 153kB 33.2MB/s 
     |████████████████████████████████| 256kB 25.6MB/s 
  Created wheel for jupyter-server-proxy: filename=jupyter_server_proxy-1.2.0-cp36-none-any.whl size=17665 sha256=d953f906fe83bb270f5751bf9fc562797670b71f08a07eace175cb66b0fcf8dd
  Stored in directory: /root/.cache/pip/wheels/c0/4b/8a/0c16c1ea15d25039985bc3cb7be3323d32fbf085a63f4e6266
  Created wheel for simpervisor: filename=simpervisor-0.3-cp36-none-any.whl size=5199 sha256=6fd17055314669d42627dee89b4f4253f8f4194679f9162e2e0be433699d0a89
  Stored in directory: /root/.cache/pip/wheels/80/36/bc/7bf2d3b5b367c5a3492dc6308304e19c48b102e39597bb90a6
  Created wheel for idna-ssl: filename=idna_ssl-1.1.0-cp36-none-any.whl size=3162 sha256=bec054bd5d6891d2b25ddf49f6d9d64da312f50963892e451630bdf4455eda11
  Stored in directory: /root/.cache/pip/wheels/d3/00/b3/32d613e19e08a739751dd6bf998cfed277728f8b2127ad4eb7
Successfully built ju

In [0]:
import jupyterlab_dash
import dash
import dash_html_components as html

viewer = jupyterlab_dash.AppViewer()

app = dash.Dash(__name__)

app.layout = html.Div('Hello World')

viewer.show(app)

In [0]:
from ipywidgets import Dropdown, Layout, HTML
from IPython.display import clear_output

dataset_dw = Dropdown(
    options = list(zip(dataset_df['DatasetName'], dataset_df['DatasetID'])),
    value = dataset_df['DatasetID'][0],
    description = 'Dataset',
    layout = Layout(margin='20px')
)

imglogs = reader.get_tray_thumb_imglogs(dataset_df['DatasetID'][0])
mosaic_html = reader.get_mosaic_image(imglogs[0].log_id)

def on_change(change):
  if change['type'] == 'change' and change['name'] == 'value':
    imglogs = reader.get_tray_thumb_imglogs(change['new'])
    mosaic_html = reader.get_mosaic_image(imglogs[0].log_id)
    clear_output(True)
    display(dataset_w, HTML(value=mosaic_html))

dataset_w.observe(on_change)

display(dataset_dw, HTML(value=mosaic_html))

Dropdown(description='Dataset', layout=Layout(margin='20px'), options=(('877270_KIND168', 'c7bfb49c-2e85-4d31-…

HTML(value='\r\n\r\n\r\n\r\n\r\n\r\n<!DOCTYPE html>\r\n<html>\r\n  <head>\r\n    <title>NVCL Data Services :: …